In [ ]:
import xml.etree.ElementTree as ET

tree = ET.parse('rocket2.xml')
root = tree.getroot()
configurations = []
motors_by_configid = {}

In [ ]:
for motor in root.findall(".//motor"):
    config_id = motor.get('configid')
    motors_by_configid[config_id] = {
      'manufacturer': motor.find('manufacturer').text,
      'designation': motor.find('designation').text,
      'delay': motor.find('delay').text
    }

print('**', motors_by_configid)


In [ ]:
for motorconfig in root.findall(".//motorconfiguration"):
  config_id = motorconfig.get('configid')
  print('**', config_id)
  stage = motorconfig.find('.//stage')
  stageNumber = int(stage.get('number')) if stage is not None else None
  stageActive = stage.get('active') == 'true' if stage is not None else False

  motor_details = motors_by_configid.get(config_id, {})
  manufacturer = motor_details.get('manufacturer', "Unknown")
  designation = motor_details.get('designation', "Unknown")
  delay = motor_details.get('delay', "Unknown")
  
  # ignitionEvent = motorconfig.find('.//ignitionEvent').text if motorconfig.find('.//ignitionEvent') else "Unknown"
  # ignitionDelay = motorconfig.find('.//ignitionDelay').text if motorconfig.find('.//ignitionDelay') else "Unknown"

  config_data = {
    'configId': config_id,
    'stageNumber': stageNumber,
    'stageActive': stageActive,
    'manufacturer': manufacturer,
    'designation': designation,
    'delay': delay,
    # 'ignitionEvent': ignitionEvent,
    # 'ignitionDelay': ignitionDelay,
    # 'simulations': simulations
  }

  print(config_data)


In [ ]:
simulations = []
     
for simulation in root.findall(".//simulation"):
  conditions = simulation.find('conditions')
  sim_configid= conditions.find('configid').text
  print(simulation)      
  if sim_configid == config_id:
    flight_data = simulation.find('.//flightdata')
    simulations.append({   
      'name': simulation.find('name').text,
      'simulator': simulation.find('simulator').text,
      'calculator': simulation.find('calculator').text,
      'flightData': flight_data
      })
    

In [ ]:
import xml.etree.ElementTree as ET
import json  # For optional JSON conversion

def parse_xml_to_json_with_correct_structure(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    configurations = []

    # First, build a dictionary of motors by their configid for easy access
    motors_by_configid = {}
    for motor in root.findall(".//motor"):
        config_id = motor.get('configid')
        if config_id:
            motors_by_configid[config_id] = {
                'manufacturer': motor.find('manufacturer').text if motor.find('manufacturer') else "Unknown",
                'designation': motor.find('designation').text if motor.find('designation') else "Unknown",
                'delay': motor.find('delay').text if motor.find('delay') else "Unknown"
            }

    for motorconfig in root.findall(".//motorconfiguration"):
        config_id = motorconfig.get('configid')

        stage = motorconfig.find('.//stage')
        stageNumber = int(stage.get('number')) if stage is not None else None
        stageActive = stage.get('active') == 'true' if stage is not None else False

        motor_details = motors_by_configid.get(config_id, {})
        manufacturer = motor_details.get('manufacturer', "Unknown")
        designation = motor_details.get('designation', "Unknown")
        delay = motor_details.get('delay', "Unknown")
        
        ignitionEvent = motorconfig.find('.//ignitionEvent').text if motorconfig.find('.//ignitionEvent') else "Unknown"
        ignitionDelay = motorconfig.find('.//ignitionDelay').text if motorconfig.find('.//ignitionDelay') else "Unknown"
                
        simulations = []
        for simulation in root.findall(".//simulation"):
            sim_configid = simulation.find('.//configid').text if simulation.find('.//configid') else None
            if sim_configid == config_id:
                flight_data = {child.tag: child.text for child in simulation.find('.//flightdata')}
                simulations.append({
                    'name': simulation.find('name').text if simulation.find('name') else "Unknown",
                    'simulator': simulation.find('simulator').text if simulation.find('simulator') else "Unknown",
                    'calculator': simulation.find('calculator').text if simulation.find('calculator') else "Unknown",
                    'flightData': flight_data
                })

        config_data = {
            'configId': config_id,
            'stageNumber': stageNumber,
            'stageActive': stageActive,
            'manufacturer': manufacturer,
            'designation': designation,
            'delay': delay,
            'ignitionEvent': ignitionEvent,
            'ignitionDelay': ignitionDelay,
            'simulations': simulations
        }

        configurations.append(config_data)

    return json.dumps(configurations, indent=4)

# Use the corrected function to parse the XML and convert to JSON
rocket_json_with_correct_structure = parse_xml_to_json_with_correct_structure('rocket2.xml')

# Print the JSON to verify the output
print(rocket_json_with_correct_structure[:500])  # Displaying the first 500 characters for brevity
